In [1]:
from genetic_algorithm import *
import re
import json
import numpy as np
from numpy.random import default_rng
from collections import Counter
import copy
import matplotlib.pyplot as plt


## hexadecimal generator

In [2]:
nr_of_input = 3
nr_of_actions = 3
nr_of_inner = 3
nr_of_genes = 16
nr_individuals = 4

individuals = generate_initial_genomes_for_population(nr_individuals, nr_of_genes, nr_of_input, nr_of_actions, nr_of_inner)

In [3]:
## world size
world_size_x = 128
world_size_y = 128

rng = default_rng()
x = rng.choice(world_size_x, size=nr_individuals, replace=False)
y = rng.choice(world_size_y, size=nr_individuals, replace=False)


In [4]:
individuals_sum_dup = sum_duplicated_neurons(individuals)

## initial brain and position generator

In [5]:
initial_dic_of_output_neurons = {}

## sum duplicates
individuals_sum_dup = sum_duplicated_neurons(individuals)

## brain generator
for individual in individuals_sum_dup:
    
    ## init brain neuron generator
    
    ## preprocessing
    edges = individuals_sum_dup[individual]
    edges = [tuple(edges[i]) for i in edges]
    edges_no_weight = [i[:-1] for i in edges]
    
    G = nx.MultiDiGraph()
    G.add_edges_from(edges)
    
    out_list = list(set([i[1] for i in edges if 'out' in i[1]]))
    init_list = list(set([i[0] for i in edges if 'in' in i[0]]))

    # generate dict of paths
    dic_of_paths = generate_dict_of_paths(out_list, init_list, G)

    lis = filtered_neurons_paths(dic_of_paths)
    
    dic = append_weight_to_neurons_in_path(lis, edges_no_weight, edges)
    print(dic)
    
        
        

#     print(dic)
    print()

{'out1': {'out1': {'in1': -3.4085151838852434, 'in2': -0.9473523576987639}}, 'out2': {'mid0': {'mid1': 1.7848313749427742, 'in1': 0.7307797955135052}, 'mid1': {'in2': 3.2488325957576683}, 'out2': {'in0': -1.6188005493667024, 'in2': 2.51707614832901, 'mid0': -1.9496413856249046}}, 'out0': {'mid0': {'mid1': 1.7848313749427742, 'in1': 0.7307797955135052}, 'mid1': {'in2': 3.2488325957576683}, 'mid2': {'mid1': 1.069800091561117, 'in0': -2.6313444224019533, 'mid0': 2.1737830001526017}, 'out0': {'mid0': -3.53779948115367, 'mid2': -0.8854570425759194, 'in2': 1.5251640470013734, 'in1': -0.7518998931786968}}}

{'out1': {'out1': {'in1': 0.0876545093850145}}, 'out2': {'out2': {'in2': -1.6151381046848772, 'in1': 1.228628109262933}}, 'out0': {'mid2': {'in1': 2.2646116282618647}, 'out0': {'in1': 0.857988707462231, 'in0': -0.26894552113535786, 'mid2': -2.5590721806806043}}}

{'out1': {'mid1': {'in0': 2.005798870746223}, 'out1': {'in0': -3.6962612543873035, 'mid1': 1.3963680756905235, 'in2': -3.0032046

In [7]:
dic['out1']

{'mid0': {'in1': 1.5150312833816573},
 'mid1': {'in0': -2.4261254387303524,
  'in2': 2.1773233633450326,
  'in1': -1.7663970700442546},
 'mid2': {'mid0': 1.5067297421028536},
 'out1': {'mid2': -0.0036624446818251184,
  'mid1': 0.10865252556081184,
  'in0': -0.30251793071875477}}

In [4]:
initial_dic_of_output_neurons = {}

## sum duplicates
individuals_sum_dup = sum_duplicated_neurons(individuals)

## remove self loops
individuals_sum_dup_no_self_loop = remove_self_loop(individuals_sum_dup)

## brain generator
for individual in individuals_sum_dup_no_self_loop:
    
    ## init brain neuron generator
    
    ## preprocessing
    edges = individuals_sum_dup_no_self_loop[individual]
    edges = [tuple(edges[i]) for i in edges]

    remove_mid_with_no_predecessor(edges) 
    dic = weight_sum_preprocessing(edges)

    ## calculate weight sum
    mid_to_weight(dic)
    out_to_weight(dic)
       
    dic = {k:v for k, v in dic.items() if 'out' in k}    
    
    initial_dic_of_output_neurons[individual] = {}
    initial_dic_of_output_neurons[individual]['output'] = dic
    
    ## position
#     initial_dic_of_output_neurons[individual]['position'] = {}
#     initial_dic_of_output_neurons[individual]['position']['x'] = x[individual]
#     initial_dic_of_output_neurons[individual]['position']['y'] = y[individual] 
    
    ## position history
    initial_dic_of_output_neurons[individual]['history_position'] = [(x[individual], y[individual])]

In [5]:
initial_dic_of_output_neurons

{0: {'output': {'out0': -0.3773868456335161,
   'out1': -0.9937793533910032,
   'out2': 0.6629433488186646},
  'history_position': [(56, 86)]},
 1: {'output': {'out0': -0.6491238145100007,
   'out1': -0.9938525052518815,
   'out2': 0.996742499582871},
  'history_position': [(108, 57)]},
 2: {'output': {'out0': -0.5770437074250989,
   'out1': 0.9996225397838105,
   'out2': -0.8923346831380427},
  'history_position': [(116, 98)]},
 3: {'output': {'out0': 0.7206053313883478,
   'out1': 0.3851296468432839,
   'out2': -0.9983551724925369},
  'history_position': [(45, 58)]},
 4: {'output': {'out0': 0.9611346334877263, 'out2': 0.9926729215935661},
  'history_position': [(18, 107)]},
 5: {'output': {'out0': -0.8415034007186649,
   'out1': -0.9945782401205644,
   'out2': 0.9984068410980266},
  'history_position': [(127, 123)]},
 6: {'output': {'out0': 0.9971004923295347,
   'out1': 0.9878636573119111,
   'out2': -0.9987185068899346},
  'history_position': [(34, 84)]},
 7: {'output': {'out0': 0.

In [25]:
a = [(3, 0),(4,1),(5,2)]

In [9]:
def slope_intercept(x1,y1,x2,y2):
    a = (y2 - y1) / (x2 - x1)
    b = y1 - a * x1     
    return a,b

print(slope_intercept(3,0,2,1))

(-1.0, 3.0)


In [21]:
a

[(3, 0), (4, 1), (5, 2)]

In [26]:
if a[-2][0] == a[-1][0]:
    x = a[-1][0]
elif a[-2][0] > a[-1][0]:
    x = a[-2][0] + 1
elif a[-2][0] < a[-1][0]:
    x = a[-2][0] - 1
print(x)

3


In [22]:
a[-2][0]

4